# FAIRNESS BENCHATON

_________________

This notebook compiles the results of the Fairness Benchaton conducted in june 2021 at datacraft. 

Currently results have been merged from : Aequitas and Fairlearn

<u>Methodology:</u>

We used the dataset about <a href='https://raw.githubusercontent.com/ekimetrics/ethical-ai-toolkit/main/data/german-risk-scoring.csv'>credit risk </a> scoring categorizing german individuals by various socio-economical attributes (gender,age,family, incomes...)  and annotated with the risk associated with each individuals as a credit carrier (high risk or low risk).<br>
This annotation is directly the judgment of a business expert which means it is not based on a ground truth (like a retroactive annotation after the fact) but rather purely reflects the mechanism of choice in the fields. <br>
This leads to the non-surprising effect that the model trained on this dataset tends to be biaised toward certain features such as age and gender to name a few , effectively realizing a discrimination. 

<u> Structure </u>

The idea is to layout the various benchmark using :
    * the same training data
    * the same models. 

1. <u>the common ground:</u> imports , class definition and such... all the titles of this section are black

2. <u style='color:green'>the load of the data:</u> , analysis and the training of two reference model for the benchmark.

2. <u style='color:brown'>Aequitas:</u> benchmark of Aequitas librairy , all the titles of this section are brown

3. <u style='color:blue'>Fairlearn:</u> benchmark of Fairlearn librairy , all the titles of this section are blue. 




The first cells are the 'common ground' where the data are loaded , analyzed and two models are trained on the training data,   a logistic regression model and a random forest.<br>
Both model information (y_predict , accuracy, training data...) are encapsulated into two object-variables: 
    <span style='color:green'> logistic_regression</span> and <span style='color:green'> random_forest </span>
    the training data are also stored in the object-variable  <span style='color:green'> training_data </span>

Two classes have been defined : 
    * ModelRun to store all the information about a model into a single variable
    * SplitData to store all the information about the training data into a single variable. 

This simplifies the chaining of benchmarks and opens the way for a possible comparison and cross-usage of the libs. 

<u style='color:green'>Details of the classes</u> 

The class ModelRun stores the information using the following structure: 
* training_data : is a SplitData Object containing X_train, X_test, Y_train....
* model : the model used (a scikit-learn Object) 
* y_predict : the predictions
* accuracy : the accuracy score
* confusion matrix : the confusion matrix

The class SplitData is just an encapsulation of the sklearn train_test_split() for the sake of storing variables in case of multiple training data (currently overkill but it makes the previous class a bit simpler). It's attributes are (logically) X_train,X_test,y_train and y_test. 



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
import warnings
print('done importing modules')

# Custom toolbox

In [ ]:
def evaluate_model(model,X_train,y_train,X_test,remove_warnings=True,display_result=True):
    if remove_warnings:
        warnings.filterwarnings('ignore')
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test,y_pred)
    matrix = confusion_matrix(y_test,y_pred)
    
    
    plt.title(f"{str(model)} - accuracy {acc:.4f}")
    sns.heatmap(matrix,annot = True,cmap = "Blues",square = True)
    plt.show()
    
    return y_test,y_pred,model

In [ ]:
class SplitData:
    def __init__(
        self,
        X,
        y,
        test_size=0.3,
        stratify=None
    ):
        stratify=y if stratify is None else stratify
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(X,y,test_size = test_size,stratify=y)
   
                

In [ ]:
class ModelRun:
    def __init__(
        self,
        model,
        X=None,
        y=None,
        training_data=None,              
        remove_warnings=True
    ):
        if not any([var is not None for var in [X,training_data]]):
            raise Exception('you must provide either training data as a SplitData() object  or X,y arguments ')
        self.training_data = SplitData(X,y,test_size = 0.3,stratify=y) if training_data is None else training_data
        self.model=model
        
        # registering training results attributes
        self.y_pred=None,
        self.accuracy=None,
        self.confusion_matrix=None     
    
    def train(self,remove_warnings=False,display_result=True):
        if remove_warnings:
            warnings.filterwarnings('ignore')  # at your own risk :)
        self.model.fit(self.training_data.X_train,self.training_data.y_train)
        self.y_pred=self.model.predict(self.training_data.X_test) 
        self.accuracy=accuracy_score(self.training_data.y_test,self.y_pred)
        self.confusion_matrix=confusion_matrix(self.training_data.y_test,self.y_pred)
        if display_result:
            self.draw_result()
            
        return
    
    def draw_result(self):
        plt.title(f"{str(self.model)} - accuracy {self.accuracy:.4f}")
        sns.heatmap(self.confusion_matrix,annot = True,cmap = "Blues",square = True)
        plt.show()
        return plt
    

#  <span style='color:green'>Data preparation and model training
##  <span style='color:green'>Loading data

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/ekimetrics/ethical-ai-toolkit/main/data/german-risk-scoring.csv")
target = 'Cost Matrix(Risk)'

data["sex"] = data["Personal status and sex"].map(lambda x : x.split(":")[0])
data[target]=data[target].replace({'Good Risk': 1, 'Bad Risk': 0})
data = data.rename(columns = {"Age in years":"age"})

##  <span style='color:green'>quick analysis of dependencies toward target

In [ ]:
for col in data.columns:
    if is_numeric_dtype(data[col]):
        # Magic dist function by seaborn
        plt.figure(figsize = (15,3))
        plt.title(col)
        sns.histplot(data = data,x = col,hue = target,multiple="stack")
        plt.show()
    else:
        # Magic catplot function by seaborn
        plt.figure(figsize = (15,3))
        plt.title(col)
        sns.countplot(data = data,x = col,hue = target)
        plt.show()

##  <span style='color:green'>recording columns by data type

In [ ]:
categoricals = [
    'Status of existing checking account',
    'Credit history',
    'Purpose',
    'Savings account/bonds',
    'Present employment since',
    'Personal status and sex', 
    'Other debtors / guarantors',
    'Property',
    'Other installment plans',
    'Housing',
    'Job',
    'Telephone',
    'foreign worker',
    'sex'
       ]
numericals = [
    'Duration in month',
    'Credit amount',
    'Installment rate in percentage of disposable income',
    'Present residence since',
    'age',
    'Number of existing credits at this bank',
    'Number of people being liable to provide maintenance for',
    
    
]

##  <span style='color:green'>train models

In [ ]:
X = pd.concat([
    pd.get_dummies(data[categoricals]),
    data[numericals],
],axis = 1)
X = X.drop(columns = ["sex_male"])
y = data[target]
X.sample()

In [ ]:
training_data=SplitData(X=X,y=y)
training_data.X_train.shape

In [ ]:
random_forest=ModelRun(
    model=RandomForestClassifier(random_state=1),
    training_data=training_data
)
random_forest.train()
random_forest.y_pred.shape

In [ ]:
logistic_regression=ModelRun(
    model=LogisticRegression(),
    training_data=training_data
)
logistic_regression.train(remove_warnings=True)
logistic_regression.y_pred.shape

We know have two objects representing our baseline trained models : <span style='color:green'>logistic_regression </span>and <span style='color:green'>random_forest</span> that we can refer to in the benchmark section. Those object contains their training data (common) refered to as <span style='color:green'>training_data</span>  (either directly or through the modelRun.training_data attribute). 

#  <span style='color:red'>Analysis using benched libs
__________________

## adding some meta information about data structure.
_____________

one last step before we go ,to use the libs it's better to have stored the name of categorical and numerical columns 

In [ ]:
cats_to_analyse = [
#     'Status of existing checking account',
#     'Credit history',
#     'Purpose',
#     'Savings account/bonds',
#     'Present employment since',
#     'Personal status and sex', 
#     'Other debtors / guarantors',
#     'Property',
#     'Other installment plans',
#     'Housing',
#     'Job',
#     'Telephone',
#     'foreign worker'
    'sex_female'
       ]

numericals_to_analyse = [
#     'Duration in month',
     'Credit amount',
#     'Installment rate in percentage of disposable income',
#     'Present residence since',
     'age'#,
#     'Number of existing credits at this bank',
#     'Number of people being liable to provide maintenance for',
]

fields_to_analyse = cats_to_analyse + numericals_to_analyse
analysis = training_data.X_test[fields_to_analyse].copy()
analysis['score'] = logistic_regression.y_pred
analysis['label_value'] = training_data.y_test

# <span style='color:brown'> AEQUITAS </span>
________________

http://www.datasciencepublicpolicy.org/projects/aequitas/

Machine Learning, AI and Data Science based predictive tools are being increasingly used in problems that can have a drastic impact on people’s lives in policy areas such as criminal justice, education, public health, workforce development and social services. Recent work has raised concerns on the risk of unintended bias in these models, affecting individuals from certain groups unfairly. While a lot of bias metrics and fairness definitions have been proposed, there is no consensus on which definitions and metrics should be used in practice to evaluate and audit these systems. Further, there has been very little empirical work done on using and evaluating these measures on real-world problems, especially in public policy.

Aequitas, an open source bias audit toolkit developed by the Center for Data Science and Public Policy at University of Chicago, can be used to audit the predictions of machine learning based risk assessment tools  to understand different types of biases, and make informed decisions about developing and deploying such systems.


## <span style='color:brown'> basics of AEQUITAS:
    
Aequitas is meant to audit your dataset using four metrics that are supposed to describe 'how biaised' are your system and which "category of impacts" your can expect:

1. Equal parity : <br>
If you want each group to be represented equally among the selected set

2. Proportional parity: <br>
If you want each group represented proportional to their representation in the overall population

3. False Positive Parity:<br>
If you want each group to have equal False Positive Rates

4. False Negative Parity:<br>
If you want each group to have equal False Negative Rates




<img src="http://www.datasciencepublicpolicy.org/wp-content/uploads/2021/04/Fairness-Short-Tree-1200x746.png" />

## <span style='color:brown'> metrics in code

https://dssg.github.io/aequitas/
    
<img src="https://dssg.github.io/aequitas/_images/preliminary_concepts.jpg">
<img src="https://dssg.github.io/aequitas/_images/metrics.jpg">

## <span style='color:brown'> tweaking data : restoring the 'sex' column
___________
the preprocessing of aequitas that we used do not handle very well booleans (the one we used probably isn't the 'right'way) so...back to a categorical

In [ ]:
analysis['sex']=analysis.sex_female.apply(lambda x:'female' if x else 'male')
cats_to_analyse.append('sex')
cats_to_analyse.remove('sex_female')
analysis.drop('sex_female',axis=1,inplace=True)
analysis.sample(n=7)

## <span style='color:brown'> using aequitas preprocessing
_______


In [ ]:
from aequitas.preprocessing import preprocess_input_df
df, _ = preprocess_input_df(analysis)
df.sample(n=2)

In [ ]:
import karmahutils as kut
for col in df.columns:
    kut.display_message(col)
    print(df[col].value_counts())

<a href="https://dssg.github.io/aequitas/api/aequitas.html#src.aequitas.preprocessing.preprocess_input_df">preprocess_input_df</a> is basically a way to assign cohort belonging to the instances of your dataset. Each row will then be described by a discretize range of value instead of column values. 
basically implements <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html">pandas.qcut</a> based on quartiles. A huge drawback is that you can not customize the number of bins that you want. It's a break in four unless your cardinality is lower. And that's it. 


In [ ]:
from aequitas.group import Group
g = Group()
xtab, _ = g.get_crosstabs(df)
xtab

<a href="https://dssg.github.io/aequitas/api/aequitas.html#src.aequitas.group.Group.get_crosstabs"> get_crosstabs</a> basically computes the metric for each basic cohort resulting from the grouping in quartiles. 

## <span style='color:brown'> plotting the audit results

In [ ]:
from aequitas.plotting import Plot
    
plt.figure(figsize = (25, 15))
aqp = Plot()
fpr_plot = aqp.plot_group_metric_all(xtab, metrics=['ppr','pprev','fnr','fpr'])

In [ ]:
fpr_plot = aqp.plot_group_metric(xtab, 'fpr', min_group_size=0.05)

In [ ]:
from aequitas.bias import Bias
group_dict = {
    'age':df['age'].unique()[1], 
    'Credit amount':df['Credit amount'].unique()[1],
    'sex' : 'female'
}
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, 
                    original_df=df, 
                    ref_groups_dict=group_dict, 
                    alpha=0.05, 
                    check_significance=False)
print('analyzing group')
print(group_dict)
bdf

In [ ]:
attrib = 'sex'
fpr_disparity = aqp.plot_disparity(
    bdf,
    group_metric='fpr_disparity',
    attribute_name=attrib
)

In [ ]:
attrib = 'age'
fpr_disparity = aqp.plot_disparity(
    bdf,
    group_metric='fpr_disparity',
    attribute_name=attrib
)

In [ ]:
attrib = 'Credit amount'
fpr_disparity = aqp.plot_disparity(bdf, group_metric='fpr_disparity', 
                                       attribute_name=attrib)

In [ ]:
from aequitas.fairness import Fairness
    
f = Fairness()
fdf = f.get_group_value_fairness(bdf)
fpr_fairness = aqp.plot_fairness_group(fdf, group_metric='fpr', title=True, min_group_size=0.05)

# <span style='color:blue'>FAIRLEARN

In [ ]:
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate
# from fairlearn.widget import FairlearnDashboard   << removed to be replaced by raiwidgets
from raiwidgets import FairnessDashboard

Since fairlearn 0.7, FairLearnDashboard class do not exists. Instead it's another dedicated microsoft project that took over (raiwidgets). 


## <span style='color:blue'> Auditing 

In [ ]:
mf = MetricFrame(accuracy_score,training_data.y_test, logistic_regression.y_pred, sensitive_features=training_data.X_test["sex_female"])
print(mf.overall)
print(mf.by_group)

In [ ]:
mf = MetricFrame(selection_rate, training_data.y_test, random_forest.y_pred, sensitive_features=training_data.X_test["sex_female"])
print(mf.overall)
print(mf.by_group)

In [ ]:
from raiwidgets import FairnessDashboard
# A_test contains your sensitive features (e.g., age, binary gender)
# y_true contains ground truth labels
# y_pred contains prediction labels
dashboard=FairnessDashboard(
    sensitive_features=training_data.X_test,
    y_true=training_data.y_test,
    y_pred={"initial model": logistic_regression.y_pred}
                 )

## <span style='color:blue'> Correcting the biais

__________________

Fairlearn implements a possible 'correction' to biaised dataset by increasing the weight of a marginal cohort by a factor that is proportional to the invert of its ratio in the dataset (link needed) 

In [ ]:
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity, ErrorRate

###  <span style='color:blue'> recreating an unmitigated model

In [ ]:
A=X['sex_female']
X_=X.drop(labels=['sex_female'], axis=1)
X_train,X_test,y_train,y_test, A_train, A_test = train_test_split(X_,y,A, test_size = 0.3,stratify=data[target])
unmitigated_predictor = LogisticRegression()
unmitigated_predictor.fit(X_train, y_train)
FairnessDashboard(
    sensitive_features=A_test,
    y_true=y_test,
    y_pred={"unmitigated": unmitigated_predictor.predict(X_test)}
)

## <span style='color:blue'> using fairlearn alternate 'less_biaised' training tools
__________

### <span style='color:blue'> grid search

this grid search method is based on

In [ ]:
sweep = GridSearch(LogisticRegression(),
                   constraints=DemographicParity(),
                   grid_size=100)

sweep.fit(X_train, y_train,
          sensitive_features=A_train)

predictors = sweep.predictors_

errors, disparities = [], []
for m in predictors:
    def classifier(X): return m.predict(X)

    error = ErrorRate()
    error.load_data(X_train, pd.Series(y_train), sensitive_features=A_train)
    disparity = DemographicParity()
    disparity.load_data(X_train, pd.Series(y_train), sensitive_features=A_train)

    errors.append(error.gamma(classifier)[0])
    disparities.append(disparity.gamma(classifier).max())

all_results = pd.DataFrame({"predictor": predictors, "error": errors, "disparity": disparities})

non_dominated = []
for row in all_results.itertuples():
    errors_for_lower_or_eq_disparity = all_results["error"][all_results["disparity"] <= row.disparity]
    if row.error <= errors_for_lower_or_eq_disparity.min():
        non_dominated.append(row.predictor)

In [ ]:
dashboard_predicted = {"unmitigated": unmitigated_predictor.predict(X_test)}
for i in range(len(non_dominated)):
    key = "dominant_model_{0}".format(i)
    value = non_dominated[i].predict(X_test)
    dashboard_predicted[key] = value


FairnessDashboard(
    sensitive_features=A_test,
    y_true=y_test,
    y_pred=dashboard_predicted
)

#### Adding weights to the positive "female" samples has increased the overall accuracy and decreased the disparity in predictions